# Collect full length news article

File for collecting full length news article by populating the SQS event queue.

## Data Overview

In [1]:
import numpy as np, pandas as pd

pd.set_option('display.max_colwidth', None)

In [27]:
df = pd.read_parquet("./data/data_domestic_news.parquet")
df.sample(n=10)

,url,headline,datePublished_site,probability,date_collected,siteName
1339661,https://slate.com/human-interest/2024/02/grief-storage-dear-prudence-advice.html,Help! My Stepdaughter Is Using My Guest Room to Store Her Grief—Literally.,None,0.573881,2024020400,Slate
963026,None,None,None,0.351000,2024012508,TheGuardian
109978,https://nypost.com/2024/01/03/news/massachusetts-kamal-family-cause-of-death-in-5-million-mansion-revealed/,Cause of death revealed after family found dead in $5M Mass. mansion,None,0.901283,2024010318,NewYorkPost(News)
1157024,https://reason.com/2024/01/29/utah-would-rather-repeal-social-media-age-check-law-than-defend-it-in-court/,Utah Would Rather Repeal Social Media Age Check Law Than Defend It In Court,2024-01-29T12:00:00,0.961911,2024013010,Reason
1146691,https://decider.com/2024/01/29/oppenheimer-florence-pugh-camera-broke-naked-filming-sex-scene-cillian-murphy/,"'Oppenheimer' Star Florence Pugh Reveals The Camera Broke While She Was ""Naked"" And Filming Sex Scene With Cillian Murphy: ""Not Ideal Timing""",None,0.917227,2024013004,NewYorkPost(News)
105638,None,39,None,0.809540,2024010316,DailyMail
864712,None,Seven of Stockholm's best spa experiences,None,0.663430,2024012220,BBCNews
43116,https://www.buzzfeednews.com/article/ellendurney/2023-breakups-prove-men-cant-handle-superstar-women,The Celebrity Breakups Of 2023 Prove That Not All Men Are Made To Be With Superstar Women,None,0.979589,2024010202,BuzzFeedNews
860505,https://www.theepochtimes.com/bright/one-mans-journey-the-art-of-healing-by-the-heart-5564103,‘One Man’s Journey’: The Art of Healing by the Heart,None,0.860954,2024012216,TheEpochTimes
1026583,https://www.bloomberg.com/news/videos/2024-01-11/why-the-price-of-money-is-going-up?srnd=premium-canada,Why the Price of Money Is Going Up,None,0.870749,2024012704,Bloomberg


In [28]:
df["url"].sample(10)

804243                https://pagesix.com/2024/01/20/celebrity-news/gabrielle-union-and-dwyane-wades-daughter-kaavia-5-sweetly-sings-happy-birthday/
268484                                               https://abcnews.go.com/GMA/Living/video/cecil-dog-eats-thousands-dollars-owners-money-106140807
1369574    https://www.dailymail.co.uk/femail/article-13029563/Wardrobe-malfunctions-Celeb-stylist-pro-tips-double-sided-tape-fashion-solutions.html
406698                                      https://thefederalist.com/2024/01/10/gypsy-rose-needs-psychological-help-not-a-fawning-spot-on-the-view/
173859                  https://www.economist.com/the-world-ahead/2023/11/13/america-will-need-a-new-vocabulary-to-discuss-its-presidential-election
1328728                                                https://www.nbcnews.com/news/nbcblk/mlkx-aims-offer-fuller-picture-two-icons-wives-rcna135869
789953                   https://www.cbsnews.com/news/convicted-killer-euren-balbuena-attacked-victims-ste

In [ ]:
# get md5 hash an unique identifier for article
s = "https://www.bloomberg.com/news/videos/2024-01-11/why-the-price-of-money-is-going-up?srnd=premium-canada"
md5_hash = hashlib.md5(s.encode()).hexdigest()
md5_hash

## Send Data to SQS Queue

Sending data to SQS for article collection

In [69]:
from tqdm import tqdm
import boto3
import uuid

In [80]:
sqs = boto3.client('sqs')
queue_url = 'news_article_collection'

messages_batch = []
for x in tqdm(df["url"].unique()):
    if len(messages_batch) >= 10:
        # send message
        response = sqs.send_message_batch(
            QueueUrl=queue_url,
            Entries=messages_batch)

        # empty the batch
        messages_batch = []

    message = {"Id" : str(uuid.uuid4()), "MessageBody" : json.dumps({"url" : x})}
    messages_batch.append(message)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124973/124973 [02:15<00:00, 920.24it/s]
